# Catalyst classification tutorial

This notebook is a modified version of [classification tutorial](https://github.com/catalyst-team/catalyst/blob/master/examples/notebooks/classification-tutorial.ipynb) authored by [Roman Tezikov](https://github.com/TezRomacH). Many thanks for the catalyst-team for their hard work!

[![Catalyst logo](https://raw.githubusercontent.com/catalyst-team/catalyst-pics/master/pics/catalyst_logo.png)](https://github.com/catalyst-team/catalyst)

## Requirements

Download and install the latest version of catalyst and other libraries required for this tutorial.

In [0]:
!pip install -U catalyst
!pip install albumentations
!pip install pretrainedmodels

### Colab extras

First of all, do not forget to change the runtime type to GPU. <br/>
To do so click `Runtime` -> `Change runtime type` -> Select `"Python 3"` and `"GPU"` -> click `Save`. <br/>
After that you can click `Runtime` -> `Run` all and watch the tutorial.


To intergate visualization library `plotly` to colab, run

In [0]:
import IPython

def configure_plotly_browser_state():
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))


IPython.get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

## Setting up GPUs
PyTorch and Catalyst versions:

In [1]:
import torch, catalyst

torch.__version__, catalyst.__version__

('1.2.0', '19.10.2')

You can also specify GPU/CPU usage for this turorial.

Available GPUs

In [2]:
from catalyst.utils import get_available_gpus

get_available_gpus()

pyarrow not available, switching to pickle. To install pyarrow, run `pip install pyarrow`.
lz4 not available, disabling compression. To install lz4, run `pip install lz4`.


[0]

In [3]:
import os
from typing import List, Tuple, Callable

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # "" - CPU, "0" - 1 GPU, "0,1" - MultiGPU

## Reproducibility first

Catalyst provides a special utils for research results reproducibility. <br/>
For example, `catalyst.utils.set_global_seed` fixes seed for all main DL frameworks (` PyTorch`, `Tensorflow`,` random` and `numpy`)

In case of CuDNN you can set deterministic mode and flag to use benchmark with
`catalyst.utils.prepare_cudnn`.

In [4]:
SEED = 42
from catalyst.utils import set_global_seed, prepare_cudnn

set_global_seed(SEED)
prepare_cudnn(deterministic=True)

## Dataset

In this tutorial we will use one of four datasets:

- [Ants / Bees dataset](https://www.dropbox.com/s/ffzfpbwzwdo9qp8/ants_bees_cleared_190806.tar.gz )
- [Stanford Dogs dataset](http://vision.stanford.edu/aditya86/ImageNetDogs/) 
- [Flowers Recognition](https://www.kaggle.com/alxmamaev/flowers-recognition)
- [Best Artworks of All Time](https://www.kaggle.com/ikarus777/best-artworks-of-all-time)

If you are on MacOS and you don’t have `wget`, you can install it with:` brew install wget`.

Let's specify the dataset by `DATASET` variable:
- `ants_bees` (~ 400 pictures for 2 classes)  – for CPU experiments
- `flowers` (~ 4k pictures for 5 classes)     – for fast GPU experiments 
- `dogs` (~ 20k pictures for 120 classes)
- `artworks` (~ 8k pictures for 50 classes)   – for GPU experiments

In [5]:
DATASET = "flowers" # "ants_bees" / "flowers" / "dogs" / "artworks"
os.environ["DATASET"] = DATASET

In [0]:
%%bash

function gdrive_download () {
  CONFIRM=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=$1" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')
  wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$CONFIRM&id=$1" -O $2
  rm -rf /tmp/cookies.txt
}

mkdir -p Images
rm -rf Images/

if [[ "$DATASET" == "ants_bees" ]]; then
    gdrive_download 1czneYKcE2sT8dAMHz3FL12hOU7m1ZkE7 ants_bees_cleared_190806.tar.gz
    tar -xf ants_bees_cleared_190806.tar.gz &>/dev/null
    mv ants_bees_cleared_190806 Images
elif [[ "$DATASET" == "flowers" ]]; then
    gdrive_download 1rvZGAkdLlbR_MEd4aDvXW11KnLaVRGFM flowers.tar.gz
    tar -xf flowers.tar.gz &>/dev/null
    mv flowers Images
elif [[ "$DATASET" == "dogs" ]]; then
    # https://www.kaggle.com/jessicali9530/stanford-dogs-dataset
    wget http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar
    tar -xf images.tar &>/dev/null
elif [[ "$DATASET" == "artworks" ]]; then
    # https://www.kaggle.com/ikarus777/best-artworks-of-all-time
    gdrive_download 1eAk36MEMjKPKL5j9VWLvNTVKk4ube9Ml artworks.tar.gz
    tar -xf artworks.tar.gz &>/dev/null
    mv artworks Images
fi

In [0]:
from pathlib import Path

ROOT = "Images/"
ALL_IMAGES = list(Path(ROOT).glob("**/*.jpg"))
ALL_IMAGES = list(filter(lambda x: not x.name.startswith("."), ALL_IMAGES))
print("Number of images:", len(ALL_IMAGES))

Let's check out the data!

In [0]:
from catalyst.utils import imread
import numpy as np

import matplotlib.pyplot as plt

def show_examples(images: List[Tuple[str, np.ndarray]]):
    _indexes = [(i, j) for i in range(2) for j in range(2)]
    
    f, ax = plt.subplots(2, 2, figsize=(16, 16))
    for (i, j), (title, img) in zip(_indexes, images):
        ax[i, j].imshow(img)
        ax[i, j].set_title(title)
    f.tight_layout()

def read_random_images(paths: List[Path]) -> List[Tuple[str, np.ndarray]]:
    data = np.random.choice(paths, size=4)
    result = []
    for d in data:
        title = f"{d.parent.name}: {d.name}"
        _image = imread(d)
        result.append((title, _image))
    
    return result

You can restart the cell below to see more examples.

In [0]:
images = read_random_images(ALL_IMAGES)
show_examples(images)

## Augmentations

For augmentation of our dataset, we will use the [albumentations library](https://github.com/albu/albumentations).  <br/>
You can view the list of available augmentations on the documentation [website](https://albumentations.readthedocs.io/en/latest/api/augmentations.html).

In [0]:
from albumentations import Compose, LongestMaxSize, PadIfNeeded
from albumentations import ShiftScaleRotate, IAAPerspective, RandomBrightnessContrast, RandomGamma, \
    HueSaturationValue, ToGray, CLAHE, JpegCompression

from albumentations import Normalize
from albumentations.pytorch import ToTensor

BORDER_CONSTANT = 0
BORDER_REFLECT = 2

def pre_transforms(image_size=224):
    # Convert the image to a square of size image_size x image_size
    # (keeping aspect ratio)
    result = [
        LongestMaxSize(max_size=image_size),
        PadIfNeeded(image_size, image_size, border_mode=BORDER_REFLECT)
    ]
    
    return result

def hard_transforms():
    result = [
        # Random shifts, stretches and turns with a 50% probability
        ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=15,
            border_mode=BORDER_REFLECT,
            p=0.5
        ),
        IAAPerspective(scale=(0.02, 0.05), p=0.3),
        # Random brightness / contrast with a 30% probability
        RandomBrightnessContrast(
            brightness_limit=0.2, contrast_limit=0.2, p=0.3
        ),
        # Random gamma changes with a 30% probability
        RandomGamma(gamma_limit=(85, 115), p=0.3),
        # Randomly changes the hue, saturation, and color value of the input image
        HueSaturationValue(p=0.3),
        JpegCompression(quality_lower=80),
    ]
    
    return result

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
    return [Normalize(), ToTensor()]

def compose(_transforms):
    # combine all augmentations into one single pipeline
    result = Compose([item for sublist in _transforms for item in sublist])
    return result


train_transforms = compose([pre_transforms(), hard_transforms(), post_transforms()])
valid_transforms = compose([pre_transforms(), post_transforms()])

show_transforms = compose([pre_transforms(), hard_transforms()])

Let's look at the augmented results. <br/>
The cell below can be restarted.

In [0]:
images = read_random_images(ALL_IMAGES)

images = [
    (title, show_transforms(image=i)["image"])
    for (title, i) in images
]
show_examples(images)

## Pytorch Dataset and DataLoaders
Torchvision has standard ImageFolder Dataset class for the following folder structure:
```
dataset/
    class_1/
        *.ext
        ...
    class_2/
        *.ext
        ...
    ...
    class_N/
        *.ext
        ...
```

Let's use it

In [0]:
from torchvision import datasets
import cv2

def read_image(path):
    image_bgr = cv2.imread(path)
    assert image_bgr is not None, f"None image {path}"
    assert image_bgr.ndim == 3
    assert image_bgr.dtype == np.uint8, "Albumentations will have incorrect normalization"
    return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

class AlbDataset(datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None,
                 loader=None, is_valid_file=None):
        # we want to override only the default loader here to read images as np.uint8 (by default torchvision reads PIL)
        if loader is None:
            loader = read_image
        super().__init__(root, transform=transform, target_transform=target_transform, loader=loader, is_valid_file=is_valid_file)

    def __getitem__(self, index):
        path, target = self.samples[index]
        image = self.loader(path)
        if self.transform is not None:
            # if you use albumentations you will have to change your dataset form torchvision's defaults
            # as self.transform is applied in different maneer
            # or modify transformation function
            # self.transform(**dict_input) -> dict_output
            image = self.transform(image=image)["image"]
        if self.target_transform is not None:
            # maybe we will need target transform (like one-hot encoding or label smoothing)
            target = self.target_transform(target)

        return image, target

In [0]:
TEST_SIZE = 0.2

def has_image_ext(filename):
    return os.path.splitext(filename)[1].lower() in (".jpg", ".jpeg", ".png", ".tif", ".tiff")

def is_test_file(filename):
    # more smart and balanced way will be to split images in each folder separately
    # but let's omit this for simplicity
    # Warning: in your HW use the best splitting strategy you know, the one implemented here is not really a good one
    if not has_image_ext(filename):
        return False
    return hash(filename) % int(1. / TEST_SIZE) == 0

def is_train_file(filename):
    return has_image_ext(filename) and not is_test_file(filename)

test_dataset = AlbDataset("Images", loader=read_image, transform=train_transforms, target_transform=None, is_valid_file=is_test_file)
train_dataset = AlbDataset("Images", loader=read_image, transform=valid_transforms, target_transform=None, is_valid_file=is_train_file)
class_names = train_dataset.classes
num_classes = len(class_names)
len(train_dataset), len(test_dataset), class_names

In [0]:
train_dataset[0]

In [0]:
from collections import OrderedDict
from torch.utils.data import DataLoader
# from torch.utils.data._utils import default_collate

# we want to obtain both `targets` and `targets_one_hot`
# the more transparent (and correct) way to do so would be to modify the dataset's __getitem__
def collate_fn(batch):
    # batch == [dataset[idx1], dataset[idx2], ..., dataset[idxN]] where N is batch size
    # in our case dataset[idxK] = (image [torch.tensor], label [int])

    features, targets = list(zip(*batch))

    features = torch.stack(features, dim=0)
    targets = torch.tensor(targets, dtype=torch.long)
    targets_onehot = torch.zeros(targets.size(0), num_classes, dtype=torch.long, device=targets.device)
    targets_onehot[torch.arange(targets.size(0), device=targets.device), targets] = 1
    return {'features': features, 'targets': targets, 'targets_one_hot': targets_onehot}


def get_loaders(batch_size=64, num_workers=4, sampler=None, **kwargs):
    # we have to use ordered dict as the iteration goes `for loader in loaders`
    # and the order is important (hope it will be fixed in the future)
    loaders = OrderedDict()
    loaders["train"] = DataLoader(train_dataset, batch_size=batch_size, shuffle=sampler is None, sampler=sampler, num_workers=num_workers, **kwargs)
    loaders["valid"] = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, sampler=None, num_workers=num_workers, **kwargs)
    return loaders

loaders = get_loaders(collate_fn=collate_fn)

## Model

Let's take the classification model from [Cadene pretrain models](https://github.com/Cadene/pretrained-models.pytorch). This repository contains a huge number of pre-trained PyTorch models. <br/>
But at first, let's check them out!

In [0]:
import pretrainedmodels

pretrainedmodels.model_names

For this tutorial purposes, `ResNet18` is good enough, but you can try other models

In [0]:
model_name = "resnet18"

By `pretrained_settings` we can see what the given network expects as input and what would be the expected output.

In [0]:
pretrainedmodels.pretrained_settings[model_name]

The model returns logits for classification into 1000 classes from ImageNet. <br/>
Let's define a function that will replace the last fully-conected layer for our number of classes.

In [0]:
from torch import nn
def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet"):
    model_fn = pretrainedmodels.__dict__[model_name]
    model = model_fn(num_classes=1000, pretrained=pretrained)
    
    dim_feats = model.last_linear.in_features
    model.last_linear = nn.Linear(dim_feats, num_classes)

    return model

## Model training

In [0]:
import torch

# model creation
model = get_model(model_name, num_classes)

# as we are working on basic classification problem (no multi-class/multi-label)
# let's use standard CE loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[9], gamma=0.3
)

To run some DL experiment, Catalyst uses a [Runner](https://catalyst-team.github.io/catalyst/api/dl.html#catalyst.dl.core.runner.Runner) abstraction. <br/>
It contains main logic about "how" you run the experiment and getting predictions.

For supervised learning case, there is an extention for Runner – [SupervisedRunner](https://catalyst-team.github.io/catalyst/api/dl.html#module-catalyst.dl.runner.supervised), which provides additional methods like `train`, `infer` and `predict_loader`.

In [0]:
from catalyst.dl.runner import SupervisedRunner

runner = SupervisedRunner()

# folder for all the experiment logs
logdir = "./logs/classification_tutorial_0"
NUM_EPOCHS = 10

Using [Callbacks](https://catalyst-team.github.io/catalyst/api/dl.html#catalyst.dl.core.callback.Callback), the basic functionality of the catalyst can be expanded.

A callback is a class inherited from `catalyst.dl.core.Callback` and implements one / several / all methods:

```
on_stage_start
    --- on_epoch_start
    ------ on_loader_start
    --------- on_batch_start
    --------- on_batch_end
    ------ on_loader_end
    --- on_epoch_end
on_stage_end

on_exception - if the code crashes with an error, you can catch it and reserve the parameters you need
```

You can find the list of standard callbacks [here](https://catalyst-team.github.io/catalyst/api/dl.html#module-catalyst.dl.callbacks.checkpoint). 
It includes callbacks such as
- CheckpointCallback. Saves N best models in logdir
- TensorboardLogger. Logs all metrics to tensorboard
- EarlyStoppingCallback. Early training stop if metrics do not improve for the `patience` of epochs
- ConfusionMatrixCallback. Plots ConfusionMatrix per epoch in tensorboard

Many different metrics for classfication
- AccuracyCallback
- MapKCallback
- AUCCallback
- F1ScoreCallback

segmentation
- DiceCallback
- IouCallback

and many other callbacks, like LRFinder and MixupCallback

In [0]:
# as we are working on classification task
from catalyst.dl.callbacks import AccuracyCallback, AUCCallback, F1ScoreCallback

In [0]:
runner.train(
    model=model,
    logdir=logdir,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    # We can specify the callbacks list for the experiment;
    # For this task, we will check accuracy, AUC and F1 metrics
    callbacks=[
        AccuracyCallback(num_classes=num_classes),
        AUCCallback(
            num_classes=num_classes,
            input_key="targets_one_hot",
            class_names=os.listdir("Images")
        ),
        F1ScoreCallback(
            input_key="targets_one_hot",
            activation="Softmax"
        )
    ],
    num_epochs=NUM_EPOCHS,
    verbose=True
)

### Training analysis and model predictions 

The `utils.plot_metrics` method reads tensorboard logs from the logdir and plots beautiful metrics with `plotly` package.

In [0]:
from catalyst.dl import utils
# it can take a while (colab issue)
utils.plot_tensorboard_log(logdir, step='batch')
# utils.plot_metrics(
#     logdir=logdir, 
#     # specify which metrics we want to plot
#     metrics=["loss", "accuracy01/_mean", "auc/_mean", "f1_score", "_base/lr"]
# )

The method below will help us look at the predictions of the model for each image.

In [0]:
from torch.nn.functional import softmax

def show_prediction(
    model: torch.nn.Module, 
    class_names: List[str], 
    titles: List[str],
    images: List[np.ndarray],
    device: torch.device
) -> None:
    with torch.no_grad():
        tensor_ = torch.stack([
            valid_transforms(image=image)["image"]
            for image in images
        ]).to(device)
        
        
        logits = model.forward(tensor_)
        probabilities = softmax(logits, dim=1)
        predictions = probabilities.argmax(dim=1)
    
    images_predicted_classes = [
        (f"predicted: {class_names[x]} | correct: {title}", image)
        for x, title, image in zip(predictions, titles, images)
    ]
    show_examples(images_predicted_classes)


In [0]:
device = utils.get_device()
titles, images = list(zip(*read_random_images(ALL_IMAGES)))
titles = list(map(lambda x: x.rsplit(":")[0], titles))
show_prediction(model, class_names=class_names, titles=titles, images=images, device=device)

### Training with Focal Loss and OneCycle

In the `catalyst.contrib` there are a large number of different additional criterions, models, layers etc

For example,

[catalyst.contrib.criterion](https://catalyst-team.github.io/catalyst/api/contrib.html#module-catalyst.contrib.criterion.ce):
- HuberLoss
- CenterLoss
- FocalLossMultiClass
- DiceLoss / BCEDiceLoss
- IoULoss / BCEIoULoss
- LovaszLossBinary / LovaszLossMultiClass / LovaszLossMultiLabel
- WingLoss

Lr scheduler in [catalyst.contrib.schedulers](https://catalyst-team.github.io/catalyst/api/contrib.html#module-catalyst.contrib.schedulers.base):
- OneCycleLRWithWarmup

Moreover, in [catalyst.contrib.models](https://catalyst-team.github.io/catalyst/api/contrib.html#models) you can find various models for segmentation:
- Unet / ResnetUnet
- Linknet / ResnetLinknet
- FPNUnet / ResnetFPNUnet
- PSPnet / ResnetPSPnet
- MobileUnet


Finally, several handwritten modules in [catalyst.contrib.modules](https://catalyst-team.github.io/catalyst/api/contrib.html#module-catalyst.contrib.modules.common):
- Flatten
- TemporalAttentionPooling
- LamaPooling
- NoisyLinear
- GlobalAvgPool2d / GlobalMaxPool2d
- GlobalConcatPool2d / GlobalAttnPool2d

a bunch of others


But for now, let's take `FocalLoss` and `OneCycleLRWithWarmup` to play around.

In [0]:
from catalyst.contrib.criterion import FocalLossMultiClass
from catalyst.contrib.schedulers import OneCycleLRWithWarmup

model = get_model(model_name, num_classes)

criterion = FocalLossMultiClass()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
scheduler = OneCycleLRWithWarmup(
    optimizer, 
    num_steps=NUM_EPOCHS,
    lr_range=(0.001, 0.0001),
    warmup_steps=1
)

# FocalLoss expects one_hot for the input
# in our Reader function we have already created the conversion of targets in one_hot
# so, all we need - respecify the target key name
runner = SupervisedRunner(input_target_key="targets_one_hot")
logdir = "./logs/classification_tutorial_1"
NUM_EPOCHS = 10

In [0]:
runner.train(
    model=model,
    logdir=logdir,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_classes),
        AUCCallback(
            num_classes=num_classes,
            input_key="targets_one_hot",
            class_names=class_names
        ),
        F1ScoreCallback(
            input_key="targets_one_hot",
            activation="Softmax"
        )
    ],
    num_epochs=NUM_EPOCHS,
    verbose=True,
)

In [0]:
# it can take a while (colab issue)
utils.plot_tensorboard_log(logdir, step='batch')
# utils.plot_metrics(
#     logdir=logdir, 
#     # specify which metrics we want to plot
#     metrics=["loss", "accuracy01/_mean", "auc/_mean", "f1_score", "_base/lr"]
# )

In [0]:
device = utils.get_device()
titles, images = list(zip(*read_random_images(ALL_IMAGES)))
titles = list(map(lambda x: x.rsplit(":")[0], titles))
show_prediction(model, class_names=class_names, titles=titles, images=images, device=device)

### Balancing classes in the dataset

There are several useful data-sampler implementations in the `catalyst.data.sampler`. For example,
- `BalanceClassSampler` allows you to create stratified sampling on an unbalanced dataset. <br/> A strategy can be either downsampling, upsampling or some prespeficied number of samples per class. <br/> Very important feature for every classification problem.
- `MiniEpochSampler` allows you to split your "very large dataset" and sample some small portion of it every epoch. <br/> This is useful for those cases where you need to check valid metrics and save checkpoints more often. <br/> For example, your 1M images dataset can be sampled in 100k per epoch with all necessary metrics.

In [0]:
from catalyst.data.sampler import BalanceClassSampler

labels = [x[1] for x in train_dataset]
sampler = BalanceClassSampler(labels, mode="upsampling")

# let's re-create our loaders with BalanceClassSampler
loader = get_loaders(collate_fn=collate_fn, sampler=sampler)

model = get_model(model_name, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
scheduler = OneCycleLRWithWarmup(
    optimizer, 
    num_steps=NUM_EPOCHS, 
    lr_range=(0.001, 0.0001),
    warmup_steps=1
)

runner = SupervisedRunner()
logdir = "./logs/classification_tutorial_2"
NUM_EPOCHS = 10

In [0]:
runner.train(
    model=model,
    logdir=logdir,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=[
        AccuracyCallback(num_classes=num_classes),
        AUCCallback(
            num_classes=num_classes,
            input_key="targets_one_hot",
            class_names=class_names
        ),
        F1ScoreCallback(
            input_key="targets_one_hot",
            activation="Softmax"
        )
    ],
    num_epochs=NUM_EPOCHS,
    verbose=True,
)

In [0]:
# it can take a while (colab issue)
utils.plot_tensorboard_log(logdir, step='batch')
# utils.plot_metrics(
#     logdir=logdir, 
#     # specify which metrics we want to plot
#     metrics=["loss", "accuracy01/_mean", "auc/_mean", "f1_score", "_base/lr"]
# )

In [0]:
device = utils.get_device()
titles, images = list(zip(*read_random_images(ALL_IMAGES)))
titles = list(map(lambda x: x.rsplit(":")[0], titles))
show_prediction(model, class_names=class_names, titles=titles, images=images, device=device)

## Model inference

With SupervisedRunner, you can easily predict entire loader with only one method call.

In [0]:
predictions = runner.predict_loader(
    model, loaders["valid"],
    resume=f"{logdir}/checkpoints/best.pth", verbose=True
)

The resulting object has shape = (number of elements in the loader, output shape from the model)

In [0]:
predictions.shape

Thus, we can obtain probabilities for our classes.

In [0]:
print("logits: ", predictions[0])

In [0]:
probabilities = torch.softmax(torch.from_numpy(predictions[0]), dim=0)
print("probabilities: ", probabilities)

In [0]:
label = probabilities.argmax().item()
print(f"predicted: {class_names[label]}")
